In [1]:
import laspy
import glob, os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
kitti_files = glob.glob('/nas2/YJ/DATA/kitti/training/velodyne/*.bin')

In [3]:
len(kitti_files)

7481

In [4]:
# This is the config in kitti_dataset.yaml
# https://github.com/open-mmlab/OpenPCDet/blob/master/tools/cfgs/dataset_configs/kitti_dataset.yaml
'''
 POINT_CLOUD_RANGE: [0, -40, -3, 70.4, 40, 1]
   - NAME: transform_points_to_voxels
      VOXEL_SIZE: [0.05, 0.05, 0.1]
      MAX_POINTS_PER_VOXEL: 5
      MAX_NUMBER_OF_VOXELS: {
        'train': 16000,
        'test': 40000
      }
'''

"\n POINT_CLOUD_RANGE: [0, -40, -3, 70.4, 40, 1]\n   - NAME: transform_points_to_voxels\n      VOXEL_SIZE: [0.05, 0.05, 0.1]\n      MAX_POINTS_PER_VOXEL: 5\n      MAX_NUMBER_OF_VOXELS: {\n        'train': 16000,\n        'test': 40000\n      }\n"

In [5]:
# if voxel size is 0.16m, psuedo image be 375*375(60/0.16)
x_voxel_size = 0.05
y_voxel_size = 0.05
z_voxel_size = 0.1
x_range_min = 0.0
x_range_max = 70.4
y_range_min = -40
y_range_max = 40
z_range_min = -3
z_range_max = 1
x_size = int((x_range_max-x_range_min)/x_voxel_size)
y_size = int((y_range_max-y_range_min)/y_voxel_size)
z_size = int((z_range_max-z_range_min)/z_voxel_size)
voxel = np.zeros((x_size, y_size, z_size))
print ('{}m, {}m and {}m of lidar ==> {}px, {}px and {}px of 3D voxel ==> {} locations'.format(
    x_range_max-x_range_min,
    y_range_max-y_range_min,
    z_range_max-z_range_min,
    voxel.shape[0],
    voxel.shape[1],
    voxel.shape[2],
    voxel.shape[0]*voxel.shape[1]*voxel.shape[2]
    ))

70.4m, 80m and 4m of lidar ==> 1408px, 1600px and 40px of 3D voxel ==> 90112000 locations


In [6]:
idxes = np.random.randint(len(kitti_files), size=10)
voxels=[]
non_zeros=[]
for idx in idxes :
    print (100*'#')
    print ('file: ', kitti_files[idx])
    points = np.fromfile(kitti_files[idx], dtype=np.float32).reshape(-1, 4)
    print ('num_of_points: {}'.format(points.shape[0]))
    print ('range_of_x: {} to {}'.format(min(points[:,0]), max(points[:,0])))
    print ('range_of_y: {} to {}'.format(min(points[:,1]), max(points[:,1])))
    print ('range_of_z: {} to {}'.format(min(points[:,2]), max(points[:,2])))
    voxel = np.zeros((x_size, y_size, z_size))
    for point in points :
        if x_range_min <= point[0] <= x_range_max \
        and y_range_min <= point[1] <= y_range_max \
        and z_range_min <= point[2] <= z_range_max:
            r = int((point[0]-x_range_min)//x_voxel_size)
            c = int((point[1]-y_range_min)//y_voxel_size)
            d = int((point[2]-z_range_min)//z_voxel_size)
            voxel[r,c,d] +=1
    voxels.append(voxel)
    non_x, non_y, non_z = np.where(voxel != 0)
    non_zeros.append(non_x.shape[0])
    print ('non-zero_voxels: {} voxel'.format(non_x.shape[0]))
    print ('non-zero_ratio: {:.2f} %'.format(100* (non_x.shape[0]/(voxel.shape[0] * voxel.shape[1] * voxel.shape[2]))))
print ('average number of non_zeros: {:.2f}'.format(sum(non_zeros)/10.0))

####################################################################################################
file:  /nas2/YJ/DATA/kitti/training/velodyne/005596.bin
num_of_points: 111678
range_of_x: -77.76899719238281 to 79.96499633789062
range_of_y: -75.26399993896484 to 77.86599731445312
range_of_z: -17.292999267578125 to 2.878000020980835
non-zero_voxels: 36626 voxel
non-zero_ratio: 0.04 %
####################################################################################################
file:  /nas2/YJ/DATA/kitti/training/velodyne/003680.bin
num_of_points: 124093
range_of_x: -79.74800109863281 to 79.9260025024414
range_of_y: -59.80099868774414 to 42.832000732421875
range_of_z: -7.328000068664551 to 2.874000072479248
non-zero_voxels: 50227 voxel
non-zero_ratio: 0.06 %
####################################################################################################
file:  /nas2/YJ/DATA/kitti/training/velodyne/004997.bin
num_of_points: 123880
range_of_x: -79.86399841308594 to 78.322998046

In [8]:
non_zeros = []
for idx in range(len(kitti_files)) :
    points = np.fromfile(kitti_files[idx], dtype=np.float32).reshape(-1, 4)
    voxel = np.zeros((x_size, y_size, z_size))
    for point in points :
        if x_range_min < point[0] < x_range_max \
        and y_range_min < point[1] < y_range_max \
        and z_range_min < point[2] < z_range_max:
            r = int((point[0]-x_range_min)//x_voxel_size)
            c = int((point[1]-y_range_min)//y_voxel_size)
            d = int((point[2]-z_range_min)//z_voxel_size)
            voxel[r,c,d] +=1
    non_x, non_y, non_z = np.where(voxel != 0)
    non_zeros.append(non_x.shape[0])
print ('average number of non_zeros: {:.2f}'.format(sum(non_zeros)/len(kitti_files)))
print ('averae ratio of non_zeros: {:.2f}'.format(100*(sum(non_zeros)/len(kitti_files))/(voxel.shape[0] * voxel.shape[1] * voxel.shape[2])))

average number of non_zeros: 39959.42
averae ratio of non_zeros: 0.04


In [9]:
non_zeros = []
for idx in range(len(kitti_files)) :
    points = np.fromfile(kitti_files[idx], dtype=np.float32).reshape(-1, 4)
    voxel = np.zeros((x_size, y_size, z_size))
    for point in points :
        if x_range_min < point[0] < x_range_max \
        and y_range_min < point[1] < y_range_max \
        and z_range_min < point[2] < z_range_max:
            r = int((point[0]-x_range_min)//x_voxel_size)
            c = int((point[1]-y_range_min)//y_voxel_size)
            d = int((point[2]-z_range_min)//z_voxel_size)
            voxel[r,c,d] +=1
    non_x, non_y, non_z = np.where(voxel != 0)
    non_zeros.append(non_x.shape[0])
print ('average number of non_zeros: {:.2f}'.format(sum(non_zeros)/len(kitti_files)))
print ('averae ratio of non_zeros: {:.2f}'.format(100*(sum(non_zeros)/len(kitti_files))/(voxel.shape[0] * voxel.shape[1] * voxel.shape[2])))

average number of non_zeros: 39949.22
averae ratio of non_zeros: 0.04
